# Optimise multiplet parameters using the standards

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore") # suppress annoying warnings

import sys
sys.path.append('/home/rstudio/codes')

from glob import glob
import os
from collections import defaultdict
import re

from IPython.display import display, HTML
import seaborn as sns

from pyBatman import PyBatmanPipeline, sub_dir_path

## 1. Setup the pipeline

In [ ]:
background_dir = '/home/rstudio/NMR/calibrations/background'
output_dir = None
database_file = '/home/rstudio/codes/databases/default_db.csv'

In [ ]:
input_backgrounds = sub_dir_path(background_dir)
pipeline = PyBatmanPipeline(input_backgrounds, 'cpmg', '.', database_file)

In [ ]:
print pipeline.tsp_range

## 2. Load spiked metabolites

In [ ]:
std_concentrations = [50, 100, 250, 500]

In [ ]:
names = sorted(pipeline.db.metabolites.keys())
lower_names = {}
for name in names:
    tokens = name.split('_')
    value = tokens[0].lower()
    lower_names[name] = value
names = sorted(set(lower_names.values()))

In [ ]:
for name in sorted(pipeline.db.metabolites.keys()):
    if name == 'TSP':
        continue
    print name, lower_names[name]

In [ ]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
base_dir = '/home/rstudio/NMR/calibrations/spiked_metabolites/'
metabolite_concentrations = defaultdict(list)
for name in sorted(pipeline.db.metabolites.keys()):
    if name == 'TSP':
        continue
    print '======================================================='
    print 'Loading %s' % name
    print '=======================================================' 
    
    ln = lower_names[name]
    paths = natural_sort(glob(os.path.join(base_dir, '%s*'%ln)))
    for input_spectra in paths:
        my_dir = os.path.basename(os.path.normpath(input_spectra))
        tokens = my_dir.split('_')
        metabolite = tokens[0]
        conc = int(tokens[1])
        pipeline.load_spiked(name, conc, input_spectra)
        metabolite_concentrations[name].append(conc)
    print
    print

## 3. Optimise model parameters

In [ ]:
tsp_concentration = 2320

In [ ]:
db = pipeline.db
for name in sorted(db.metabolites.keys()):
    # do not optimise TSP    
    if name == 'TSP':
        continue        
    # a new copy of the db is created each time with the corrected relative intensity
    if name in metabolite_concentrations:
        std_concentrations = metabolite_concentrations[name]
        db = pipeline.update_rel_intensities(db, name, std_concentrations, tsp_concentration)

In [ ]:
display(db.df)

## 4. Save the DB

In [ ]:
db.df.to_csv('/home/rstudio/codes/databases/default_optimised_db.csv', index=False)